In [1]:
import pandas as pd
import numpy as np
import requests
import shutil
import gzip
import io

#### Realizando download do Arquivo

In [2]:
url  = 'https://data.brasil.io/dataset/covid19/caso_full.csv.gz'
local= 'C:/Users/fabio.pedrosa/Documents/pessoal/Laboratorio_02/analise_covid_19/fabio'
s = requests.Session()
r = s.get(url)
with gzip.open(io.BytesIO(r.content), 'r') as f_in, open('caso_full.csv', 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)

#### Lendo o dataset 

In [3]:
df = pd.read_csv('caso_full.csv',sep=',',parse_dates=['date']).sort_values('city_ibge_code')
df.dropna(inplace=True)
df.head(5)

,city,city_ibge_code,date,epidemiological_week,estimated_population,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths
189387,Alta Floresta D'Oeste,1100015.0,2020-06-10,24,22728.0,22945.0,False,True,19,83.59732,2020-06-08,0.0000,0,40,city,RO,0,0
673345,Alta Floresta D'Oeste,1100015.0,2020-09-07,37,22728.0,22945.0,False,False,608,2675.11440,2020-09-07,0.0164,10,129,city,RO,9,0
869798,Alta Floresta D'Oeste,1100015.0,2020-10-12,42,22728.0,22945.0,False,False,841,3700.28159,2020-10-12,0.0119,10,164,city,RO,0,0
123249,Alta Floresta D'Oeste,1100015.0,2020-05-26,22,22728.0,22945.0,False,False,3,13.19958,2020-05-26,0.0000,0,25,city,RO,0,0
1049510,Alta Floresta D'Oeste,1100015.0,2020-11-13,46,22728.0,22945.0,False,False,922,4056.67019,2020-11-13,0.0130,12,196,city,RO,5,0


In [4]:
df_municipio = pd.read_csv('municipio.csv',sep=',').sort_values('codigo_uf')
df_municipio.head(5)

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf
3406,1100502,Novo Horizonte do Oeste,-11.6961,-61.9951,0,11
4627,1101484,São Felipe D'Oeste,-11.9023,-61.5026,0,11
1428,1100072,Corumbiara,-12.9551,-60.8947,0,11
4039,1101476,Primavera de Rondônia,-11.8295,-61.3153,0,11
5193,1101559,Teixeirópolis,-10.9056,-62.2420,0,11


In [5]:
df_estado = pd.read_csv('estado.csv',sep=',').sort_values('codigo_uf')
df_estado.rename(columns={"latitude": "latitude_uf", "longitude": "longitude_uf",'nome':'nome_uf'},inplace=True)
df_estado.head(5)

,codigo_uf,uf,nome_uf,latitude_uf,longitude_uf
0,11,RO,Rondônia,-10.83,-63.34
1,12,AC,Acre,-8.77,-70.55
2,13,AM,Amazonas,-3.47,-65.10
3,14,RR,Roraima,1.99,-61.33
4,15,PA,Pará,-3.79,-52.48


#### Juntando os dataframe de estado e município

In [6]:
temp=df_municipio.merge(df_estado, left_on='codigo_uf', right_on='codigo_uf',how='inner')
colunas = ['capital','codigo_ufcodigo_uf']
temp=temp.sort_values('codigo_ibge')
temp.head(5)

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,uf,nome_uf,latitude_uf,longitude_uf
32,1100015,Alta Floresta D'Oeste,-11.92830,-61.9953,0,11,RO,Rondônia,-10.83,-63.34
31,1100023,Ariquemes,-9.90571,-63.0325,0,11,RO,Rondônia,-10.83,-63.34
37,1100031,Cabixi,-13.49450,-60.5520,0,11,RO,Rondônia,-10.83,-63.34
19,1100049,Cacoal,-11.43430,-61.4562,0,11,RO,Rondônia,-10.83,-63.34
34,1100056,Cerejeiras,-13.18700,-60.8168,0,11,RO,Rondônia,-10.83,-63.34


#### Jutando os dados do covid-19 com o data frame de município

In [7]:
temp=df.merge(temp, left_on='city_ibge_code', right_on='codigo_ibge',how='inner')
temp.head(5)

,city,city_ibge_code,date,epidemiological_week,estimated_population,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,...,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,uf,nome_uf,latitude_uf,longitude_uf
0,Alta Floresta D'Oeste,1100015.0,2020-06-10,24,22728.0,22945.0,False,True,19,83.59732,...,1100015,Alta Floresta D'Oeste,-11.9283,-61.9953,0,11,RO,Rondônia,-10.83,-63.34
1,Alta Floresta D'Oeste,1100015.0,2020-09-07,37,22728.0,22945.0,False,False,608,2675.11440,...,1100015,Alta Floresta D'Oeste,-11.9283,-61.9953,0,11,RO,Rondônia,-10.83,-63.34
2,Alta Floresta D'Oeste,1100015.0,2020-10-12,42,22728.0,22945.0,False,False,841,3700.28159,...,1100015,Alta Floresta D'Oeste,-11.9283,-61.9953,0,11,RO,Rondônia,-10.83,-63.34
3,Alta Floresta D'Oeste,1100015.0,2020-05-26,22,22728.0,22945.0,False,False,3,13.19958,...,1100015,Alta Floresta D'Oeste,-11.9283,-61.9953,0,11,RO,Rondônia,-10.83,-63.34
4,Alta Floresta D'Oeste,1100015.0,2020-11-13,46,22728.0,22945.0,False,False,922,4056.67019,...,1100015,Alta Floresta D'Oeste,-11.9283,-61.9953,0,11,RO,Rondônia,-10.83,-63.34


In [8]:
colunas = ['city', 'city_ibge_code', 'epidemiological_week',
       'estimated_population_2019', 'is_last', 'is_repeated',
       'last_available_confirmed_per_100k_inhabitants', 'last_available_date',
       'last_available_death_rate',  'order_for_place',
       'place_type', 'state', 'codigo_ibge']
temp.drop(columns=colunas,axis=1,inplace=True)
temp.describe()

,estimated_population,last_available_confirmed,last_available_deaths,new_confirmed,new_deaths,latitude,longitude,capital,codigo_uf,latitude_uf,longitude_uf
count,1.275587e+06,1.275587e+06,1.275587e+06,1.275587e+06,1.275587e+06,1.275587e+06,1.275587e+06,1.275587e+06,1.275587e+06,1.275587e+06,1.275587e+06
mean,4.434115e+04,6.706359e+02,2.062390e+01,5.790615e+00,1.489698e-01,-1.626587e+01,-4.614097e+01,6.132863e-03,3.214501e+01,-1.625171e+01,-4.658429e+01
std,2.548616e+05,4.932665e+03,2.279341e+02,5.097441e+01,1.865365e+00,8.370703e+00,6.472831e+00,7.807212e-02,9.828114e+00,8.254930e+00,6.357560e+00
min,7.760000e+02,1.000000e+00,0.000000e+00,-9.060000e+02,-1.110000e+02,-3.368660e+01,-7.289970e+01,0.000000e+00,1.100000e+01,-3.017000e+01,-7.055000e+01
25%,5.965000e+03,2.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,-2.282040e+01,-5.077530e+01,0.000000e+00,2.500000e+01,-2.219000e+01,-5.155000e+01
50%,1.304500e+04,9.100000e+01,2.000000e+00,0.000000e+00,0.000000e+00,-1.781430e+01,-4.648250e+01,0.000000e+00,3.100000e+01,-1.810000e+01,-4.544000e+01
75%,2.870400e+04,3.480000e+02,7.000000e+00,2.000000e+00,0.000000e+00,-8.108710e+00,-4.112400e+01,0.000000e+00,4.100000e+01,-8.380000e+00,-4.171000e+01
max,1.232523e+07,3.924010e+05,1.547700e+04,1.934500e+04,4.380000e+02,4.603140e+00,-3.241070e+01,1.000000e+00,5.300000e+01,1.990000e+00,-3.659000e+01


In [9]:
temp['ativos']= (temp.last_available_confirmed - temp.last_available_deaths) + temp.new_confirmed
temp.head(5)

,date,estimated_population,last_available_confirmed,last_available_deaths,new_confirmed,new_deaths,nome,latitude,longitude,capital,codigo_uf,uf,nome_uf,latitude_uf,longitude_uf,ativos
0,2020-06-10,22728.0,19,0,0,0,Alta Floresta D'Oeste,-11.9283,-61.9953,0,11,RO,Rondônia,-10.83,-63.34,19
1,2020-09-07,22728.0,608,10,9,0,Alta Floresta D'Oeste,-11.9283,-61.9953,0,11,RO,Rondônia,-10.83,-63.34,607
2,2020-10-12,22728.0,841,10,0,0,Alta Floresta D'Oeste,-11.9283,-61.9953,0,11,RO,Rondônia,-10.83,-63.34,831
3,2020-05-26,22728.0,3,0,0,0,Alta Floresta D'Oeste,-11.9283,-61.9953,0,11,RO,Rondônia,-10.83,-63.34,3
4,2020-11-13,22728.0,922,12,5,0,Alta Floresta D'Oeste,-11.9283,-61.9953,0,11,RO,Rondônia,-10.83,-63.34,915


#### Obtem o numero de casos confirmados, mortes, recuperados e ativos agrupado por data

In [10]:
import warnings
warnings.filterwarnings('ignore')

temp_agg = temp.groupby(['date'])['last_available_confirmed','last_available_deaths','new_confirmed', 'new_deaths', 'ativos'].sum().reset_index()
temp_agg.head(5)

,date,last_available_confirmed,last_available_deaths,new_confirmed,new_deaths,ativos
0,2020-02-25,1,0,1,0,2
1,2020-02-26,1,0,0,0,1
2,2020-02-27,1,0,0,0,1
3,2020-02-28,2,0,1,0,3
4,2020-02-29,2,0,0,0,2


#### Remodela o dataframe com variável e valor para ter quantidades de recuperados, mortos e ativos

In [11]:
temp_01 = temp_agg.melt(id_vars="date", value_vars=['new_confirmed', 'new_deaths', 'ativos'],
                 var_name='casos', value_name='Quantidade')
temp_01.head(5)

,date,casos,Quantidade
0,2020-02-25,new_confirmed,1
1,2020-02-26,new_confirmed,0
2,2020-02-27,new_confirmed,0
3,2020-02-28,new_confirmed,1
4,2020-02-29,new_confirmed,0


In [12]:
# habilita modo offline
from plotly.offline import plot, iplot, init_notebook_mode
import plotly.express as px
init_notebook_mode(connected=True)

In [13]:
# Definindo o renderizador para abrir numa nova aba:
#import plotly.io as pio
#pio.renderers
#pio.renderers.default = "browser"

In [14]:
# Cores
new_confirmed = '#21bf73'
new_deaths = '#ff2e63'
ativos = '#fe9801'

#### Plotando o gráfico de quantida de casos ao longo do tempo

In [15]:
fig = px.area(temp_01, 
              x="date", 
              y="Quantidade", 
              color='casos', 
              height=600,
              title='Casos ao longo do tempo',
              color_discrete_sequence = [new_confirmed, new_deaths, ativos])
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

#### Agrupando os dados de covid-19 por data e cidade

In [16]:
temp_agg = temp.groupby(['date', 'uf','codigo_uf','latitude_uf','longitude_uf'])['new_confirmed', 'new_deaths', 'ativos'].sum().reset_index()
temp_agg.head(5)

,date,uf,codigo_uf,latitude_uf,longitude_uf,new_confirmed,new_deaths,ativos
0,2020-02-25,SP,35,-22.19,-48.79,1,0,2
1,2020-02-26,SP,35,-22.19,-48.79,0,0,1
2,2020-02-27,SP,35,-22.19,-48.79,0,0,1
3,2020-02-28,SP,35,-22.19,-48.79,1,0,3
4,2020-02-29,SP,35,-22.19,-48.79,0,0,2


In [17]:
import numpy as np
temp_agg.date.describe()
temp_agg.rename(columns={'latitude_uf':'latitude','longitude_uf':'longitude'},inplace=True)
temp_agg.sort_values('ativos', ascending=False)

,date,uf,codigo_uf,latitude,longitude,new_confirmed,new_deaths,ativos
7731,2020-12-24,SP,35,-22.19,-48.79,19733,363,1392256
7812,2020-12-27,SP,35,-22.19,-48.79,7685,105,1387788
7839,2020-12-28,SP,35,-22.19,-48.79,1576,39,1383216
7758,2020-12-25,SP,35,-22.19,-48.79,0,0,1372523
7785,2020-12-26,SP,35,-22.19,-48.79,0,0,1372523
...,...,...,...,...,...,...,...,...
32,2020-03-10,DF,53,-15.83,-47.86,0,0,1
212,2020-03-21,MT,51,-12.64,-55.42,0,0,1
30,2020-03-10,AL,27,-9.62,-36.82,0,0,1
27,2020-03-09,MG,31,-18.10,-44.38,0,0,1


#### Plotando o gráfico de casos no decorrer do tempo

In [18]:
fig = px.density_mapbox(temp_agg,
                        title='Casos ao longo do tempo',
                        lat='latitude', 
                        lon='longitude', 
                        z=temp_agg["ativos"],
                        #radius=10,
                        hover_name='uf',
                        hover_data=["new_confirmed", "new_deaths"],
                        center={"lat": -15.83, "lon": -47.86}, 
                        zoom=3,
                        color_continuous_scale="Viridis",
                        animation_frame=temp_agg["date"].dt.strftime('%Y-%m-%d'),
                        mapbox_style="open-street-map"
                       )
fig.update_layout(autosize=False, width=1200, height=800)   
fig.show()

In [19]:
import json

with open('br_states.json') as f:
    estados = json.load(f)
#counties

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(go.Choroplethmapbox(geojson=estados,
                                    locations=temp_agg.uf,
                                    z=temp_agg.ativos,
                                    colorscale="Viridis", 
                                    marker_opacity=0.5, 
                                    marker_line_width=0))
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=3, mapbox_center = {"lat": -15.83, "lon": -47.86})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()